In [570]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import joblib

from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import spacy
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

import statistics


import re
from os import path
from PIL import Image
import plotly
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.collocations import *
nltk.download('averaged_perceptron_tagger')

from textblob import TextBlob

import seaborn as sns

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hasee\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [571]:
def normalize_list(data):
	x_min = min(data)
	x_max = max(data)
	if x_max-x_min == 0:
		return None
	return [(x-x_min)/(x_max-x_min) for x in data]

In [572]:
df = pd.read_csv('../../data_with_senti/ge_sentiments.csv', encoding='utf-8')

In [573]:
df.head()

,Unnamed: 0,Company,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
0,0,GE,5.0,0,NaN,Good company,2022-02-25,Technical Product Manager,"Detroit, MI",2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,NaN,0.44,Competitive salary and training opportunitiesE...,-0.08
1,1,GE,4.0,0,NaN,Its electric,2022-02-24,Anonymous Employee,NaN,0,0,0,"lots of power, lots of light","to bright sometimes, less power",NaN,0.00,"lots of power, lots of lightto bright sometime...",0.44
2,2,GE,5.0,1,less than 1 year,Good Company to start with,2022-02-24,Lead Engineer,"Harlem, FL",-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",NaN,0.44,"Everything, you've s job, be happyNothing, act...",0.20
3,3,GE,4.0,0,more than 5 years,great experience,2022-02-23,Manager Total Rewards,"Los Angeles, CA",0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,NaN,0.62,"loved the team, the flexibility to choose spec...",0.84
4,4,GE,4.0,1,more than 5 years,Great place for experience,2022-02-21,Gas Turbine Technician,"Houston, TX",0,0,0,Job security and lots of OT,Raises aren’t good and management expects loyalty,NaN,0.62,Job security and lots of OTRaises aren’t good ...,0.83


In [574]:
df.columns

Index(['Unnamed: 0', 'Company', 'Star_Rating', 'Current_Employee', 'Work_Duration', 'Review_Title', 'Date_Posted', 'Job_Title', 'Job_Location', 'Recommend', 'CEO_Approval', 'Business_Outlook', 'Pros', 'Cons', 'Advice_to_Management', 'Review_Title_sentiment_score_vader', 'Pros_And_Cons', 'Pros_And_Cons_sentiment_score_vader'], dtype='object')

In [575]:
df.drop(['Unnamed: 0', 'Company'], axis=1, inplace=True)

In [576]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
0,5.0,0,NaN,Good company,2022-02-25,Technical Product Manager,"Detroit, MI",2,0,1,Competitive salary and training opportunities,Extreme work load and unclear responsibility.,NaN,0.44,Competitive salary and training opportunitiesE...,-0.08
1,4.0,0,NaN,Its electric,2022-02-24,Anonymous Employee,NaN,0,0,0,"lots of power, lots of light","to bright sometimes, less power",NaN,0.00,"lots of power, lots of lightto bright sometime...",0.44
2,5.0,1,less than 1 year,Good Company to start with,2022-02-24,Lead Engineer,"Harlem, FL",-1,1,-1,"Everything, you've s job, be happy","Nothing, actually. Everything is fine.",NaN,0.44,"Everything, you've s job, be happyNothing, act...",0.20
3,4.0,0,more than 5 years,great experience,2022-02-23,Manager Total Rewards,"Los Angeles, CA",0,2,0,"loved the team, the flexibility to choose spec...",does not offer stock as part of the comp packa...,NaN,0.62,"loved the team, the flexibility to choose spec...",0.84
4,4.0,1,more than 5 years,Great place for experience,2022-02-21,Gas Turbine Technician,"Houston, TX",0,0,0,Job security and lots of OT,Raises aren’t good and management expects loyalty,NaN,0.62,Job security and lots of OTRaises aren’t good ...,0.83


In [577]:
df['Date_Posted'] = pd.to_datetime(df['Date_Posted'], dayfirst=True)

In [578]:
quarters = pd.Series(pd.date_range('2012-01-01', '2022-01-01', freq='QS'))

In [579]:
quarters

0    2012-01-01
1    2012-04-01
2    2012-07-01
3    2012-10-01
4    2013-01-01
5    2013-04-01
6    2013-07-01
7    2013-10-01
8    2014-01-01
9    2014-04-01
10   2014-07-01
11   2014-10-01
12   2015-01-01
13   2015-04-01
14   2015-07-01
15   2015-10-01
16   2016-01-01
17   2016-04-01
18   2016-07-01
19   2016-10-01
20   2017-01-01
21   2017-04-01
22   2017-07-01
23   2017-10-01
24   2018-01-01
25   2018-04-01
26   2018-07-01
27   2018-10-01
28   2019-01-01
29   2019-04-01
30   2019-07-01
31   2019-10-01
32   2020-01-01
33   2020-04-01
34   2020-07-01
35   2020-10-01
36   2021-01-01
37   2021-04-01
38   2021-07-01
39   2021-10-01
40   2022-01-01
dtype: datetime64[ns]

In [580]:
df = df[(df['Date_Posted'] >= '2012-01-01') & (df['Date_Posted'] < '2021-12-31')]

In [581]:
for i, row in df.iterrows():
	if isinstance(row['Work_Duration'], str):
		if 'less' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0] - 1
			# row['Work_Duration'] = years[0] - 1
		elif 'more' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0]
			# row['Work_Duration'] = years[0]

In [582]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader
108,4.0,0,10,Great place to work,2021-12-30,Legal Counsel,NaN,2,2,2,Lots of opportunities to grow and learn; solvi...,Large organization makes it hard to act quickl...,NaN,0.62,Lots of opportunities to grow and learn; solvi...,0.49
111,4.0,1,NaN,good,2021-12-28,Director,"San Jose, CA",2,2,2,people with solid tech skills,typical corporate with slow decision,NaN,0.44,people with solid tech skillstypical corporate...,0.15
113,5.0,0,NaN,Great Company,2021-12-27,Warehouse Manager,"Saginaw, TX",2,2,2,Benefits and pay is really good,overworked too many hours no home life,NaN,0.62,Benefits and pay is really goodoverworked too ...,0.00
114,5.0,1,3,Great place to work,2021-12-28,Engineer,"Richmond, VA",2,2,2,Get to work on technical products,None that I can think of,NaN,0.62,Get to work on technical productsNone that I c...,0.00
115,4.0,1,NaN,Overall a good place to work,2021-12-27,Finance Director,"Chicago, IL",0,0,0,Flexible work environment with high-caliber co...,"Large company, things can move slowly.",NaN,0.44,Flexible work environment with high-caliber co...,0.23


In [583]:
df.drop('Job_Title', axis=1, inplace=True)

In [584]:
df.drop('Advice_to_Management', axis=1, inplace=True)

In [585]:
df.drop('Job_Location', axis=1, inplace=True)

In [586]:
df['Work_Duration'].fillna(df['Work_Duration'].mean(),inplace=True)

# other value aggregates

In [587]:
star_rating_aggregates = dict()
title_senti_aggregates = dict()
pnc_senti_aggregates = dict()
work_duration_aggregates = dict()

current_employee_aggregates = dict()
recommend_aggregates = dict()
ceo_approval_aggregates = dict()
business_outlook_aggregates = dict()



# percentages
recommend_percentages = dict()
ceo_percentages = dict()
business_percentages = dict()
counts = dict()
count = 0

In [588]:

for q in quarters:
	star_rating_aggregates[f'Q{count+1}'] = 0
	title_senti_aggregates[f'Q{count+1}'] = 0
	pnc_senti_aggregates[f'Q{count+1}'] = 0
	work_duration_aggregates[f'Q{count+1}'] = 0


	current_employee_aggregates[f'Q{count+1}'] = []
	recommend_aggregates[f'Q{count+1}'] = []
	ceo_approval_aggregates[f'Q{count+1}'] = []
	business_outlook_aggregates[f'Q{count+1}'] = []

	recommend_percentages[f'Q{count+1}'] = dict()
	ceo_percentages[f'Q{count+1}'] = dict()
	business_percentages[f'Q{count+1}'] = dict()
	for i in range(-1, 3):
		recommend_percentages[f'Q{count+1}'][i] = 0 
		ceo_percentages[f'Q{count+1}'][i] = 0
		business_percentages[f'Q{count+1}'][i] = 0


	counts[f'Q{count+1}'] = 0
	count += 1

for idx, row in df.iterrows():
	for i in range(len(quarters)):
		if i < len(quarters) - 1:
			if row['Date_Posted'] >= quarters[i] and row['Date_Posted'] < quarters[i+1]:
				star_rating_aggregates[f'Q{i+1}'] += row['Star_Rating']
				title_senti_aggregates[f'Q{i+1}'] += row['Review_Title_sentiment_score_vader']
				pnc_senti_aggregates[f'Q{i+1}'] += row['Pros_And_Cons_sentiment_score_vader']
				work_duration_aggregates[f'Q{i+1}'] += row['Work_Duration']


				current_employee_aggregates[f'Q{i+1}'].append(row['Current_Employee'])
				recommend_aggregates[f'Q{i+1}'].append(row['Recommend'])
				ceo_approval_aggregates[f'Q{i+1}'].append(row['CEO_Approval'])
				business_outlook_aggregates[f'Q{i+1}'].append(row['Business_Outlook'])

				recommend_percentages[f'Q{i+1}'][row['Recommend']] += 1
				ceo_percentages[f'Q{i+1}'][row['CEO_Approval']] += 1
				business_percentages[f'Q{i+1}'][row['Business_Outlook']] += 1
				
				counts[f'Q{i+1}'] += 1

In [589]:

avg_star_ratings = []
avg_title_senti = []
avg_pnc_senti = []
avg_work_duration = []

avg_current_employee = []
avg_recommend = []
avg_ceo_approval = []
avg_business_outlook = []

# percentages 

recommend_minus_percentages = []
ceo_minus_percentages = []
business_minus_percentages = []

recommend_unknown_percentages = []
ceo_unknown_percentages = []
business_unknown_percentages = []

recommend_same_percentages = []
ceo_same_percentages = []
business_same_percentages = []

recommend_plus_percentages = []
ceo_plus_percentages = []
business_plus_percentages = []


quarts = []


In [590]:
for quarter, value in star_rating_aggregates.items():

	if counts[quarter] != 0:

		avg_star_ratings.append(star_rating_aggregates[quarter] / counts[quarter])
		avg_title_senti.append(title_senti_aggregates[quarter] / counts[quarter])
		avg_pnc_senti.append(pnc_senti_aggregates[quarter] / counts[quarter])
		avg_work_duration.append(work_duration_aggregates[quarter] / counts[quarter])


		avg_current_employee.append(statistics.mode(current_employee_aggregates[quarter]))
		avg_recommend.append(statistics.mode(recommend_aggregates[quarter]))
		avg_ceo_approval.append(statistics.mode(ceo_approval_aggregates[quarter]))
		avg_business_outlook.append(statistics.mode(business_outlook_aggregates[quarter]))

		recommend_minus_percentages.append(recommend_percentages[quarter][-1] / counts[quarter])
		ceo_minus_percentages.append(ceo_percentages[quarter][-1] / counts[quarter])
		business_minus_percentages.append(business_percentages[quarter][-1] / counts[quarter])

		recommend_unknown_percentages.append(recommend_percentages[quarter][0] / counts[quarter])
		ceo_unknown_percentages.append(ceo_percentages[quarter][0] / counts[quarter])
		business_unknown_percentages.append(business_percentages[quarter][0] / counts[quarter])

		recommend_same_percentages.append(recommend_percentages[quarter][1] / counts[quarter])
		ceo_same_percentages.append(ceo_percentages[quarter][1] / counts[quarter])
		business_same_percentages.append(business_percentages[quarter][1] / counts[quarter])

		recommend_plus_percentages.append(recommend_percentages[quarter][2] / counts[quarter])
		ceo_plus_percentages.append(ceo_percentages[quarter][2] / counts[quarter])
		business_plus_percentages.append(business_percentages[quarter][2] / counts[quarter])
		
		quarts.append(quarter)

<hr>

In [591]:
def clean_text(text):
	text=re.sub(r"\?", ".", text)
	text=re.sub(r"\!", ".", text)
	text=re.sub(r'([.])\1+', r'\1', text)
	rexp=r"\.(?=[A-Za-z]{1,15})"
	text=re.sub(rexp, ". ", text)
	return text

#this function extracts lemmas of nouns, verbs and adjectives
def lemma_words_pos_filtered(text):
    word_list=[]
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    for pos in pos_tokens:
        if (pos[1].startswith('N')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.NOUN)]
        if (pos[1].startswith('V')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.VERB)]
        if (pos[1].startswith('J')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.ADJ)]
    word_list=[word for word in word_list if word not in  
                                    stopwords.words('english') ]
    return  " ".join(word_list)

#to keep only unique tuples
def unique_tuples(list_of_tuples):
     list_ = [tuple(sorted(t)) for t in list_of_tuples]
     list_ = list(set(list_))
     return list_

def findbigramsintext(text):
	mylist=nltk.word_tokenize(text)
	list1=[x for x in mylist]
	feature_list = []
	for i in range(length_features2):
	# In each iteration, add an empty list to the main list      
			feature_list.append([])
	i=0
	# bigram_features is the list of unique set of bigrams found after collocation
	for l in bigram_features:
			list2=[x for x in l]
			result =  all(elem in list1  for elem in list2)
			if result: 
				feature_list[i].append(' '.join(list2))
			i=i+1
	return feature_list


def keepnonempty(list1):
       mylist= [x for x in list1 if x != []]
       return mylist
def flatten_list(row_list):
        l = [item for inner_list in row_list for item in inner_list]
        return l


# text cleaning

In [592]:

df['Cleaned_Pros_and_Cons_Text'] = df['Pros_And_Cons'].apply(sent_tokenize)

In [593]:
len(df)

10783

In [594]:
df = df.explode('Cleaned_Pros_and_Cons_Text', ignore_index=False)

In [595]:
len(df)

22314

In [596]:
df['lemma_words_pos_filtered'] = df['Cleaned_Pros_and_Cons_Text'].apply(lemma_words_pos_filtered)

In [597]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words( " ".join(df['lemma_words_pos_filtered']).split(),window_size=5)
#keeping bigrams that occur at least 5 times
finder.apply_freq_filter(5)
bigram_list_tuples=finder.nbest(bigram_measures.likelihood_ratio, 20)

In [598]:
bigram_list_tuples

[('life', 'balance'),
 ('work', 'life'),
 ('none', 'none'),
 ('work', 'balance'),
 ('red', 'tape'),
 ('work-life', 'balance'),
 ('long', 'term'),
 ('job', 'security'),
 ('n', 'n'),
 ('great', 'place'),
 ('smart', 'people'),
 ('long', 'hour'),
 ('decision', 'making'),
 ('work/life', 'balance'),
 ('large', 'company'),
 ('flexible', 'hour'),
 ('short', 'term'),
 ('leadership', 'program'),
 ('upper', 'management'),
 ('work', 'environment')]

In [599]:
topic_polarities

{'balance life': [0.022103174603174607,
  0.43514209133945975,
  0.40095598845598845,
  0.09787698412698412,
  0.6475,
  -0.09285714285714285,
  0.2955371900826446,
  0.12892405459554218,
  0.35748602426380205,
  0.44780158730158726,
  0.19769095102428436,
  0.10663690476190478,
  0.3624,
  0.3908040438397581,
  0.3130864340685769,
  0.3710273654916512,
  0.39832227891156463,
  0.35162509448223733,
  0.23882731637361268,
  0.1282589285714286,
  0.27593537414965985,
  0.19452748383303942,
  0.34542200854700855,
  0.03868452380952381,
  0.3173665490887713,
  0.2774743609565038,
  0.40675015460729746,
  0.3201851851851852,
  0.4339502164502164,
  0.3118492063492063,
  0.16947128851540616,
  0.3807391534391534,
  0.2673076923076923,
  0.30598232323232327,
  0.3058300395256917,
  0.3010662427849927,
  0.37551711389477344,
  0.2840090944720574,
  0.18934794372294372],
 'balance work': [0.06411706349206349,
  0.4015072983384387,
  0.38657395382395376,
  0.09787698412698412,
  0.49315221088435

In [600]:
bigram_features=unique_tuples(bigram_list_tuples)
length_features2=len(bigram_features)

In [601]:
df['bigram_list'] = df['lemma_words_pos_filtered'].apply(findbigramsintext)

In [602]:
df['bigrams']=df['bigram_list'].apply(keepnonempty)
df['bigrams']=df['bigrams'].apply(flatten_list)

In [603]:
#Sentiment scores are calculated using libraries TextBlob and Vader
df['polarity'] = df['Cleaned_Pros_and_Cons_Text'].apply(lambda x: TextBlob(x).sentiment[0])

In [604]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Review_Title_sentiment_score_vader,Pros_And_Cons,Pros_And_Cons_sentiment_score_vader,Cleaned_Pros_and_Cons_Text,lemma_words_pos_filtered,bigram_list,bigrams,polarity
108,4.0,0,10.00000,Great place to work,2021-12-30,2,2,2,Lots of opportunities to grow and learn; solvi...,Large organization makes it hard to act quickl...,0.62,Lots of opportunities to grow and learn; solvi...,0.49,Lots of opportunities to grow and learn; solvi...,lot opportunity grow learn solve hard problems...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],-0.083333
111,4.0,1,4.41768,good,2021-12-28,2,2,2,people with solid tech skills,typical corporate with slow decision,0.44,people with solid tech skillstypical corporate...,0.15,people with solid tech skillstypical corporate...,people solid tech skillstypical corporate slow...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],-0.100000
113,5.0,0,4.41768,Great Company,2021-12-27,2,2,2,Benefits and pay is really good,overworked too many hours no home life,0.62,Benefits and pay is really goodoverworked too ...,0.00,Benefits and pay is really goodoverworked too ...,benefit pay goodoverworked many hour home life,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.350000
114,5.0,1,3.00000,Great place to work,2021-12-28,2,2,2,Get to work on technical products,None that I can think of,0.62,Get to work on technical productsNone that I c...,0.00,Get to work on technical productsNone that I c...,get work technical productsnone think,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.000000
115,4.0,1,4.41768,Overall a good place to work,2021-12-27,0,0,0,Flexible work environment with high-caliber co...,"Large company, things can move slowly.",0.44,Flexible work environment with high-caliber co...,0.23,Flexible work environment with high-caliber co...,flexible work environment high-caliber colleag...,"[[], [], [environment work], [], [], [], [], [...",[environment work],-0.300000


In [605]:
copy_df = df.copy()

In [606]:
copy_df = copy_df.explode('bigrams', ignore_index=False)

In [607]:
res = copy_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

In [608]:
print(res)

               bigrams  polarity
0         balance life  0.293603
1         balance work  0.293349
2    balance work-life  0.314958
3    balance work/life  0.180440
4        company large  0.201659
5      decision making  0.136152
6     environment work  0.323554
7        flexible hour  0.216476
8          great place  0.545003
9            hour long  0.100451
10        job security  0.197660
11  leadership program  0.269868
12           life work  0.304188
13           long term  0.122219
14    management upper  0.097975
15                 n n  0.282992
16           none none  0.127214
17        people smart  0.276455
18            red tape  0.140912
19          short term  0.110511


In [609]:
all_bigrams = res['bigrams'].tolist()
print(all_bigrams)

['balance life', 'balance work', 'balance work-life', 'balance work/life', 'company large', 'decision making', 'environment work', 'flexible hour', 'great place', 'hour long', 'job security', 'leadership program', 'life work', 'long term', 'management upper', 'n n', 'none none', 'people smart', 'red tape', 'short term']


In [610]:
result = copy_df[['bigrams', 'polarity']].groupby('bigrams')['polarity'].mean()

In [611]:
topic_polarities = dict()
for b in all_bigrams:
	topic_polarities[b] = []

In [612]:
len(quarters)

41

In [613]:
polarity_results = []
for i in range(len(quarters)):
	if i < len(quarters) - 1:
		qtr_start = quarters[i]
		qtr_end = quarters[i+1]
		cur_df = df[(df['Date_Posted'] >= qtr_start) & (df['Date_Posted'] < qtr_end)]
		if(len(cur_df) == 0):
			continue
		cur_df = cur_df.explode('bigrams', ignore_index=False)
		results = cur_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

		for bigram in all_bigrams:
			if bigram in results['bigrams'].tolist():
				polarity = results[results['bigrams'] == bigram]['polarity'].tolist()[0]
				topic_polarities[bigram].append(polarity)
				# polarity_results.append(results[results['bigrams'] == bigram]['polarity'].tolist()[0])
			else:
				topic_polarities[bigram].append(None)

		# for i, row in results.iterrows():
		# 	if row['bigrams'] in topic_polarities:
		# 		topic_polarities[row['bigrams']].append(row['polarity'])
		# 	else:
		# 		topic_polarities[row['bigrams']] = [row['polarity']]
		polarity_results.append(results)


In [614]:
len(topic_polarities)

20

In [615]:
len(polarity_results)

40

In [616]:
for key, value in topic_polarities.items():
	print(key, len(value))

balance life 40
balance work 40
balance work-life 40
balance work/life 40
company large 40
decision making 40
environment work 40
flexible hour 40
great place 40
hour long 40
job security 40
leadership program 40
life work 40
long term 40
management upper 40
n n 40
none none 40
people smart 40
red tape 40
short term 40


In [617]:
print(counts)

{'Q1': 81, 'Q2': 78, 'Q3': 121, 'Q4': 128, 'Q5': 166, 'Q6': 131, 'Q7': 151, 'Q8': 177, 'Q9': 230, 'Q10': 241, 'Q11': 252, 'Q12': 206, 'Q13': 259, 'Q14': 344, 'Q15': 431, 'Q16': 357, 'Q17': 422, 'Q18': 398, 'Q19': 413, 'Q20': 392, 'Q21': 466, 'Q22': 422, 'Q23': 352, 'Q24': 351, 'Q25': 281, 'Q26': 217, 'Q27': 188, 'Q28': 148, 'Q29': 148, 'Q30': 177, 'Q31': 250, 'Q32': 194, 'Q33': 171, 'Q34': 160, 'Q35': 252, 'Q36': 348, 'Q37': 571, 'Q38': 411, 'Q39': 365, 'Q40': 333, 'Q41': 0}


In [618]:
final = pd.DataFrame(
{   'quarters': quarts,
	'ratings': normalize_list(avg_star_ratings),
	'title_senti': normalize_list(avg_title_senti),
	'pnc_senti': normalize_list(avg_pnc_senti),
	'work_duration': normalize_list(avg_work_duration),
	'current_employee': avg_current_employee,
	'recommend': avg_recommend,
	'ceo_approval': avg_ceo_approval,
	'business_outlook': avg_business_outlook,
	'recommend_minus': normalize_list(recommend_minus_percentages),
	'ceo_minus': normalize_list(ceo_minus_percentages),
	'business_minus': normalize_list(business_minus_percentages),
	'recommend_unknown': normalize_list(recommend_unknown_percentages),
	'ceo_unknown': normalize_list(ceo_unknown_percentages),
	'business_unknown': normalize_list(business_unknown_percentages),
	'recommend_same': normalize_list(recommend_same_percentages),
	'ceo_same': normalize_list(ceo_same_percentages),
	'business_same': normalize_list(business_same_percentages),
	'recommend_plus': normalize_list(recommend_plus_percentages),
	'ceo_plus': normalize_list(ceo_plus_percentages),
	'business_plus': normalize_list(business_plus_percentages),
})

In [619]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.569771,0.954499,0.965373,0.505268,1,2,2,0,0.144995,0.463219,0.000000,0.339864,0.000000,1.000000,None,0.814697,0.000000,0.728025,1.000000,0.000000
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.401010,0.130483,0.612179,None,0.606526,0.341880,0.564629,0.924653,0.335772
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.064248,0.183559,0.000000,None,0.567534,0.991736,0.762177,0.752186,0.715944
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.000000,0.170621,0.000150,None,0.277032,1.000000,0.870440,0.946186,0.645312
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.042974,0.179782,0.039794,None,0.482458,0.867470,1.000000,0.934141,0.849545
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.105422,0.209202,0.023048,None,0.779714,0.916031,0.677906,0.601541,0.676671
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.093881,0.240821,0.021460,None,0.674722,0.830022,0.874324,0.766894,0.653755
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.122461,0.181487,0.060028,None,0.658678,0.828625,0.663875,0.602093,0.694309
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.310959,0.233238,0.043938,None,0.578343,0.892754,0.623916,0.826217,0.779576
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.065653,0.237989,0.034472,None,0.490937,0.840941,0.913437,0.820907,0.802510


In [620]:
all_bigrams

['balance life',
 'balance work',
 'balance work-life',
 'balance work/life',
 'company large',
 'decision making',
 'environment work',
 'flexible hour',
 'great place',
 'hour long',
 'job security',
 'leadership program',
 'life work',
 'long term',
 'management upper',
 'n n',
 'none none',
 'people smart',
 'red tape',
 'short term']

In [621]:
for key,value in topic_polarities.items():
	col_name = key.replace(' ', '_')
	final[col_name] = value

In [622]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term
0,Q1,0.569771,0.954499,0.965373,0.505268,1,2,2,0,0.144995,0.463219,0.000000,0.339864,0.000000,1.000000,None,0.814697,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,NaN,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,NaN,0.000000,NaN,0.192976,0.093333,0.108929
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.401010,0.130483,0.612179,None,0.606526,0.341880,0.564629,0.924653,0.335772,0.435142,0.401507,0.377778,NaN,0.176346,NaN,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,NaN,NaN,0.508306,0.016667,0.078571
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.064248,0.183559,0.000000,None,0.567534,0.991736,0.762177,0.752186,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,NaN,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,NaN,-1.000000,0.330586,0.062500,NaN
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.000000,0.170621,0.000150,None,0.277032,1.000000,0.870440,0.946186,0.645312,0.097877,0.097877,NaN,NaN,0.179857,NaN,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,NaN,-0.110000,0.439596,NaN,NaN
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.042974,0.179782,0.039794,None,0.482458,0.867470,1.000000,0.934141,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,NaN,0.229449,0.145833,0.447823,0.250556,NaN,0.227183,0.533160,0.483333,-0.009259,NaN,NaN,0.303163,0.189983,0.000000
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.105422,0.209202,0.023048,None,0.779714,0.916031,0.677906,0.601541,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,NaN,0.317708,0.136364,-0.254762,0.000000,NaN,NaN,0.447619,0.231664,NaN
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.093881,0.240821,0.021460,None,0.674722,0.830022,0.874324,0.766894,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,NaN,NaN,0.182041,0.300379,0.122743,0.051042
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.122461,0.181487,0.060028,None,0.658678,0.828625,0.663875,0.602093,0.694309,0.128924,0.128924,NaN,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,NaN,0.115000,0.373214,0.048258,NaN
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.310959,0.233238,0.043938,None,0.578343,0.892754,0.623916,0.826217,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,NaN,0.109722,0.491667,0.288232,0.212875,0.267262,NaN,0.200000,0.355685,0.099582,0.144286
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.065653,0.237989,0.034472,None,0.490937,0.840941,0.913437,0.820907,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,NaN,0.193228,0.413597,0.101927,0.143750,NaN,0.168287,0.249233,0.297321,0.080000


In [623]:
for key, value in topic_polarities.items():
	col = key.replace(' ', '_')
	final[col].fillna(final[col].mean(),inplace=True)

In [624]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term
0,Q1,0.569771,0.954499,0.965373,0.505268,1,2,2,0,0.144995,0.463219,0.000000,0.339864,0.000000,1.000000,None,0.814697,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.215032,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.097160,0.000000,0.113819,0.192976,0.093333,0.108929
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.401010,0.130483,0.612179,None,0.606526,0.341880,0.564629,0.924653,0.335772,0.435142,0.401507,0.377778,0.213763,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.113819,0.508306,0.016667,0.078571
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.064248,0.183559,0.000000,None,0.567534,0.991736,0.762177,0.752186,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.215032,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.107754
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.000000,0.170621,0.000150,None,0.277032,1.000000,0.870440,0.946186,0.645312,0.097877,0.097877,0.340674,0.213763,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.164369,0.107754
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.042974,0.179782,0.039794,None,0.482458,0.867470,1.000000,0.934141,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.191655,0.227183,0.533160,0.483333,-0.009259,0.286658,0.113819,0.303163,0.189983,0.000000
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.105422,0.209202,0.023048,None,0.779714,0.916031,0.677906,0.601541,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.191655,0.317708,0.136364,-0.254762,0.000000,0.286658,0.113819,0.447619,0.231664,0.107754
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.093881,0.240821,0.021460,None,0.674722,0.830022,0.874324,0.766894,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.097160,0.286658,0.182041,0.300379,0.122743,0.051042
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.122461,0.181487,0.060028,None,0.658678,0.828625,0.663875,0.602093,0.694309,0.128924,0.128924,0.340674,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.107754
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.310959,0.233238,0.043938,None,0.578343,0.892754,0.623916,0.826217,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.104921,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.065653,0.237989,0.034472,None,0.490937,0.840941,0.913437,0.820907,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.191655,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000


In [625]:
earnings = [
-3843,
1257,
-1131,
-2802,
2636,
-1144,
-1987,
6199,
728,
-9423,
127,
3588,
761,
-22770,
800,
-1147,
-10820,
1360,
1057,
-83,
2336,
2027,
2908,
228,
6301,
2506,
-1360,
-13570,
5152,
3537,
3545,
2999,
3206,
3191,
3133,
3527,
4011,
3491,
3105,
3034
]

earnings = earnings[::-1]

In [626]:
final['earnings'] = earnings

In [627]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings
0,Q1,0.569771,0.954499,0.965373,0.505268,1,2,2,0,0.144995,0.463219,0.000000,0.339864,0.000000,1.000000,None,0.814697,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.215032,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.097160,0.000000,0.113819,0.192976,0.093333,0.108929,3034
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.401010,0.130483,0.612179,None,0.606526,0.341880,0.564629,0.924653,0.335772,0.435142,0.401507,0.377778,0.213763,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.113819,0.508306,0.016667,0.078571,3105
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.064248,0.183559,0.000000,None,0.567534,0.991736,0.762177,0.752186,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.215032,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.107754,3491
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.000000,0.170621,0.000150,None,0.277032,1.000000,0.870440,0.946186,0.645312,0.097877,0.097877,0.340674,0.213763,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.164369,0.107754,4011
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.042974,0.179782,0.039794,None,0.482458,0.867470,1.000000,0.934141,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.191655,0.227183,0.533160,0.483333,-0.009259,0.286658,0.113819,0.303163,0.189983,0.000000,3527
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.105422,0.209202,0.023048,None,0.779714,0.916031,0.677906,0.601541,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.191655,0.317708,0.136364,-0.254762,0.000000,0.286658,0.113819,0.447619,0.231664,0.107754,3133
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.093881,0.240821,0.021460,None,0.674722,0.830022,0.874324,0.766894,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.097160,0.286658,0.182041,0.300379,0.122743,0.051042,3191
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.122461,0.181487,0.060028,None,0.658678,0.828625,0.663875,0.602093,0.694309,0.128924,0.128924,0.340674,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.107754,3206
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.310959,0.233238,0.043938,None,0.578343,0.892754,0.623916,0.826217,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.104921,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.065653,0.237989,0.034472,None,0.490937,0.840941,0.913437,0.820907,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.191655,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545


In [628]:
pred_column = [None]
for i in range(1, len(earnings)):
	if earnings[i] > earnings[i-1]:
		pred_column.append(1)
	else:
		pred_column.append(0)

In [629]:
pred_column

[None,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0]

In [630]:
final['pred_column'] = pred_column

In [631]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings,pred_column
0,Q1,0.569771,0.954499,0.965373,0.505268,1,2,2,0,0.144995,0.463219,0.000000,0.339864,0.000000,1.000000,None,0.814697,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.215032,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.097160,0.000000,0.113819,0.192976,0.093333,0.108929,3034,NaN
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.401010,0.130483,0.612179,None,0.606526,0.341880,0.564629,0.924653,0.335772,0.435142,0.401507,0.377778,0.213763,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.113819,0.508306,0.016667,0.078571,3105,1.0
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.064248,0.183559,0.000000,None,0.567534,0.991736,0.762177,0.752186,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.215032,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.107754,3491,1.0
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.000000,0.170621,0.000150,None,0.277032,1.000000,0.870440,0.946186,0.645312,0.097877,0.097877,0.340674,0.213763,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.164369,0.107754,4011,1.0
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.042974,0.179782,0.039794,None,0.482458,0.867470,1.000000,0.934141,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.191655,0.227183,0.533160,0.483333,-0.009259,0.286658,0.113819,0.303163,0.189983,0.000000,3527,0.0
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.105422,0.209202,0.023048,None,0.779714,0.916031,0.677906,0.601541,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.191655,0.317708,0.136364,-0.254762,0.000000,0.286658,0.113819,0.447619,0.231664,0.107754,3133,0.0
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.093881,0.240821,0.021460,None,0.674722,0.830022,0.874324,0.766894,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.097160,0.286658,0.182041,0.300379,0.122743,0.051042,3191,1.0
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.122461,0.181487,0.060028,None,0.658678,0.828625,0.663875,0.602093,0.694309,0.128924,0.128924,0.340674,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.107754,3206,1.0
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.310959,0.233238,0.043938,None,0.578343,0.892754,0.623916,0.826217,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.104921,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999,0.0
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.065653,0.237989,0.034472,None,0.490937,0.840941,0.913437,0.820907,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.191655,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545,1.0


In [632]:
final.drop(['recommend_same'], axis=1, inplace=True)

In [633]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings,pred_column
0,Q1,0.569771,0.954499,0.965373,0.505268,1,2,2,0,0.144995,0.463219,0.000000,0.339864,0.000000,1.000000,0.814697,0.000000,0.728025,1.000000,0.000000,0.022103,0.064117,0.300000,0.392271,0.072436,0.275000,0.185764,0.215032,0.448165,0.152427,0.275000,0.800000,0.041973,0.108929,0.097160,0.000000,0.113819,0.192976,0.093333,0.108929,3034,NaN
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.401010,0.130483,0.612179,0.606526,0.341880,0.564629,0.924653,0.335772,0.435142,0.401507,0.377778,0.213763,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.113819,0.508306,0.016667,0.078571,3105,1.0
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.064248,0.183559,0.000000,0.567534,0.991736,0.762177,0.752186,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.215032,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.107754,3491,1.0
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.000000,0.170621,0.000150,0.277032,1.000000,0.870440,0.946186,0.645312,0.097877,0.097877,0.340674,0.213763,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.164369,0.107754,4011,1.0
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.042974,0.179782,0.039794,0.482458,0.867470,1.000000,0.934141,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.191655,0.227183,0.533160,0.483333,-0.009259,0.286658,0.113819,0.303163,0.189983,0.000000,3527,0.0
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.105422,0.209202,0.023048,0.779714,0.916031,0.677906,0.601541,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.191655,0.317708,0.136364,-0.254762,0.000000,0.286658,0.113819,0.447619,0.231664,0.107754,3133,0.0
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.093881,0.240821,0.021460,0.674722,0.830022,0.874324,0.766894,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.097160,0.286658,0.182041,0.300379,0.122743,0.051042,3191,1.0
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.122461,0.181487,0.060028,0.658678,0.828625,0.663875,0.602093,0.694309,0.128924,0.128924,0.340674,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.107754,3206,1.0
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.310959,0.233238,0.043938,0.578343,0.892754,0.623916,0.826217,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.104921,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999,0.0
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.065653,0.237989,0.034472,0.490937,0.840941,0.913437,0.820907,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.191655,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545,1.0


In [634]:
final.dropna(inplace=True)

In [635]:
final.drop(['recommend_unknown', 'ceo_unknown', 'business_unknown'], axis=1, inplace=True)

In [636]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,balance_life,balance_work,balance_work-life,balance_work/life,company_large,decision_making,environment_work,flexible_hour,great_place,hour_long,job_security,leadership_program,life_work,long_term,management_upper,n_n,none_none,people_smart,red_tape,short_term,earnings,pred_column
1,Q2,0.461594,0.675262,0.942595,0.477441,1,2,2,0,0.267683,0.486264,0.103920,0.606526,0.341880,0.564629,0.924653,0.335772,0.435142,0.401507,0.377778,0.213763,0.176346,0.145165,0.249630,0.139062,0.391042,0.367143,0.126081,0.015774,0.320556,0.037985,-0.010988,0.286658,0.113819,0.508306,0.016667,0.078571,3105,1.0
2,Q3,0.425918,0.773928,0.897899,0.000000,1,2,2,1,0.470934,0.780839,0.357279,0.567534,0.991736,0.762177,0.752186,0.715944,0.400956,0.386574,0.438476,1.000000,0.136905,0.250000,0.227247,0.215032,0.466159,-0.100000,0.600000,0.274312,0.311883,-0.050000,-0.034722,0.286658,-1.000000,0.330586,0.062500,0.107754,3491,1.0
3,Q4,0.266704,0.827978,0.347529,0.373471,1,2,2,1,0.421391,0.947365,0.443284,0.277032,1.000000,0.870440,0.946186,0.645312,0.097877,0.097877,0.340674,0.213763,0.179857,0.145165,0.191970,0.201918,0.555815,0.098034,0.124705,0.278125,0.083514,0.162500,0.161699,0.286658,-0.110000,0.439596,0.164369,0.107754,4011,1.0
4,Q5,0.588633,0.833791,0.780301,0.276686,1,2,2,2,0.201770,0.490547,0.211596,0.482458,0.867470,1.000000,0.934141,0.849545,0.647500,0.493152,0.159439,0.047024,0.214456,0.145165,0.229449,0.145833,0.447823,0.250556,0.191655,0.227183,0.533160,0.483333,-0.009259,0.286658,0.113819,0.303163,0.189983,0.000000,3527,0.0
5,Q6,0.348993,0.895306,0.590667,0.358847,1,2,2,1,0.521317,0.605006,0.433133,0.779714,0.916031,0.677906,0.601541,0.676671,-0.092857,0.171429,0.378373,0.190000,0.167159,0.000000,0.435677,-0.025000,0.570270,0.073095,0.191655,0.317708,0.136364,-0.254762,0.000000,0.286658,0.113819,0.447619,0.231664,0.107754,3133,0.0
6,Q7,0.633734,0.790989,0.773682,0.528533,1,2,2,2,0.290950,0.231376,0.554699,0.674722,0.830022,0.874324,0.766894,0.653755,0.295537,0.295354,0.229242,0.700000,0.169478,0.130093,0.150208,0.228571,0.686667,-0.020833,0.483810,0.214167,0.270909,0.028571,0.097160,0.286658,0.182041,0.300379,0.122743,0.051042,3191,1.0
7,Q8,0.322421,0.498108,0.629464,0.263410,1,2,2,2,0.516083,1.000000,0.412158,0.658678,0.828625,0.663875,0.602093,0.694309,0.128924,0.128924,0.340674,0.107407,0.130218,-0.025000,0.423231,0.700000,0.515556,0.212500,0.305556,0.150042,0.184847,0.226389,0.243810,0.286658,0.115000,0.373214,0.048258,0.107754,3206,1.0
8,Q9,0.408722,0.964106,0.701467,0.294482,1,2,2,2,0.313945,0.316219,0.270192,0.578343,0.892754,0.623916,0.826217,0.779576,0.357486,0.339708,0.500000,0.152500,0.160043,-0.100000,-0.013437,0.125000,0.500000,0.104921,0.109722,0.491667,0.288232,0.212875,0.267262,0.286658,0.200000,0.355685,0.099582,0.144286,2999,0.0
9,Q10,0.548412,1.000000,0.813475,0.214362,1,2,2,2,0.279762,0.496960,0.313916,0.490937,0.840941,0.913437,0.820907,0.802510,0.447802,0.412168,0.404911,-0.025794,0.191313,0.071154,0.286831,-0.088889,0.445432,0.067349,0.191655,0.193228,0.413597,0.101927,0.143750,0.286658,0.168287,0.249233,0.297321,0.080000,3545,1.0
10,Q11,0.522389,0.857775,0.779167,0.204886,1,2,2,2,0.264098,0.634409,0.321658,0.474655,0.698413,0.656300,0.724655,0.823442,0.197691,0.166296,0.040762,0.127976,0.161714,0.365278,0.198710,0.087500,0.479877,0.024843,0.189394,0.800000,0.218552,0.061111,-0.050000,0.286658,0.113819,0.325442,0.290278,0.128571,3537,0.0


In [638]:
final.to_csv('final_ge_cleaned_data_csv.csv')